In [258]:
import pandas as pd

df = pd.read_csv('https://github.com/AFAskar/aqar-scraper/raw/refs/heads/main/aqar_fm_listings.csv')
df

HTTPError: HTTP Error 404: Not Found

## Based on this dataset I can see that we have difference in type of property, so that will affect the prices.

In [ ]:
df['district']

0             حي النرجس
1             حي الملقا
2             حي العليا
3         حي درة المنسك
4       حي ابها الجديدة
             ...       
6255          حي النرجس
6256         حي اشبيلية
6257           حي قرطبة
6258          حي العارض
6259          حي الرمال
Name: district, Length: 6260, dtype: object

In [ ]:
df['property_type'] = df['title'].str.split().str[0]
df.groupby('property_type').count()

,id,title,url,price,meter_price,price_2_payments,price_4_payments,price_12_payments,rnpl_monthly_price,area_sqm,...,deed_number,rega_licensed,plan_no,parcel_no,user_verified,company_name,user_paid_tier,description,images,videos
property_type,,,,,,,,,,,,,,,,,,,,,
أرض,426,426,426,426,425,0,0,0,0,426,...,422,426,391,421,426,361,396,420,426,426
استراحة,67,67,67,67,0,2,2,2,0,67,...,67,67,52,57,67,66,67,66,67,67
بيت,2,2,2,2,0,0,0,0,0,2,...,0,2,0,0,2,2,0,2,2,2
دور,843,843,843,843,0,37,36,36,71,843,...,842,843,839,840,834,820,831,842,843,843
شقة,3317,3317,3317,3317,1,523,505,507,800,3317,...,3316,3317,2907,2949,3308,3168,3224,3305,3317,3317
عمارة,86,86,86,86,0,1,1,1,0,86,...,85,86,57,72,85,76,79,79,86,86
غرفة,25,25,25,25,1,9,9,9,0,25,...,25,25,20,20,25,21,22,24,25,25
فيلا,1407,1407,1407,1407,0,44,42,42,79,1407,...,1405,1407,1386,1394,1398,1353,1370,1403,1407,1407
محل,33,33,33,33,0,0,0,0,0,33,...,33,33,17,19,32,29,31,33,33,33


In [ ]:
df_sale = df[['area_sqm', "num_bedrooms", "num_bathrooms", "price", "property_type", 'city', 'district']]
df_sale = df_sale.dropna()

df_sale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3418 entries, 0 to 6257
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   area_sqm       3418 non-null   int64  
 1   num_bedrooms   3418 non-null   float64
 2   num_bathrooms  3418 non-null   float64
 3   price          3418 non-null   float64
 4   property_type  3418 non-null   object 
 5   city           3418 non-null   object 
 6   district       3418 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 213.6+ KB


In [ ]:


X = df_sale[['area_sqm', "num_bedrooms", "num_bathrooms", "property_type", "city", "district"]]
y = df_sale['price']
X

,area_sqm,num_bedrooms,num_bathrooms,property_type,city,district
0,116,4.0,3.0,دور,الرياض,حي النرجس
1,700,7.0,5.0,فيلا,الرياض,حي الملقا
6,127,4.0,3.0,شقة,جدة,حي البوادي
7,736,2.0,3.0,شقة,الرياض,حي المهدية
9,50,1.0,1.0,شقة,الرياض,حي العليا
...,...,...,...,...,...,...
6243,360,6.0,3.0,دور,الرياض,حي قرطبة
6253,100,1.0,1.0,شقة,الرياض,حي القيروان
6254,100,3.0,2.0,شقة,الرياض,حي الروضة
6256,129,4.0,2.0,شقة,الرياض,حي اشبيلية


## Split the data into Train (80%) and Test (20%)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

## Encoding categorical feature:
- Some columns are categorical text, like ('property_type', 'city', 'district')
- So we must convert categorical to numerical values so models can deal with it

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_col = ['property_type', 'city', 'district']

encoder  = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_encoded = encoder.fit_transform(X_train[cat_col])
X_test_encoded = encoder.transform(X_test[cat_col])


In [ ]:
encoded_cols = encoder.get_feature_names_out(cat_col)

X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoded_cols, index=X_train.index)
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoded_cols, index=X_test.index)

In [ ]:
X_train.columns

Index(['area_sqm', 'num_bedrooms', 'num_bathrooms', 'property_type', 'city',
       'district'],
      dtype='object')

In [ ]:
X_train = X_train.drop(columns=['property_type', 'city', 'district'])
X_test = X_test.drop(columns=['property_type', 'city','district'])

In [ ]:
X_train_final = pd.concat([X_train, X_train_encoded], axis=1)
X_test_final = pd.concat([X_test, X_test_encoded], axis=1)

In [ ]:
X_train_final

,area_sqm,num_bedrooms,num_bathrooms,property_type_استراحة,property_type_بيت,property_type_دور,property_type_شقة,property_type_فيلا,city_ابها,city_ابو عريش,...,district_حي محاسن البلدية,district_حي مخطط الدخل المحدود,district_حي مخطط الرواف,district_حي مخطط الموسى,district_حي مدينة العمال,district_حي مذينب,district_حي مريخ,district_حي مشرفة,district_حي مطار الملك خالد الدولي,district_حي هجر
3772,152,4.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3929,176,5.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3003,325,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4828,148,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5710,62,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,168,5.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1954,400,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2333,197,5.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1467,390,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model selection:
- We select LinearRegression() and RandomForestRegression() models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor



linear_model = LinearRegression()
rForest_model = RandomForestRegressor(random_state=42)


## We use Cross-validation to measure how well each model performs

In [ ]:
from sklearn.model_selection import cross_val_score

models = {
    'Linear Regression' : linear_model,
    'RandomForestRegression' : rForest_model,
}

for name, model in models.items():
    scores = cross_val_score(model, X_train_final,y_train, cv=5)
    print(name, scores.mean())

Linear Regression 0.015525651552898711
RandomForestRegression 0.5302361085232166


## We can see the RandomForestRegression is the best, so we will select it as our final model

In [ ]:
best_model = rForest_model

best_model.fit(X_train_final,y_train)

best_model.score(X_test_final,y_test)

0.4534181324704498

## Model Error Evaluation(MAE, MSE, RMSE)
- We evaluate th best_model performance using common regression error metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
y_pred = best_model.predict(X_test_final)

MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test,y_pred)

RMSE = np.sqrt(MSE)

print("MAE", MAE)
print("MSE", MSE)
print('RMSE', RMSE)

MAE 253480.75394126214
MSE 353101382328.85333
RMSE 594223.3438100975


In [ ]:
import joblib
joblib.dump(best_model, "RandomForest_model.pkl")

['RandomForest_model.pkl']

In [259]:
model = joblib.load('RandomForest_model.pkl')
model

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

## Final observation:
- The dataset contains 3418 samples and 6 features
- Random Forest Regression was better than Linear Regression
- The model achieved 45% on the test data